In [1]:
import numpy as np
import copy
import numpy as np
import random
import os 
import sys, getopt
import tensorflow as tf
import joblib

    
def create_bags(X, y, sparsity, bag_size, num_bags):
    nines = []
    notnines = []
    for i, curry in enumerate(y):
        if curry == 9:
            nines.append(i)
        else:
            notnines.append(i)
    
    bag_idxs = []
    bag_label = []
    
    numnotnine = int((1-sparsity)*bag_size)
    numnine = int(sparsity*bag_size)
    
    idxs_notnine = np.random.choice(notnines, num_bags*bag_size - int(numnine*num_bags/2), replace = True)
    idxs_nine = np.random.choice(nines, int(numnine*num_bags/2), replace = True)
    
    
    notninecount = 0
    ninecount = 0
    
    
    for i in range(num_bags):
        if i % 2 == 0:
            total_idxs = list(idxs_notnine[notninecount:notninecount + numnotnine])
            total_idxs.extend(list(idxs_nine[ninecount:ninecount + numnine]))
            
            bag_idxs.append(total_idxs)
            bag_label.append(1)
            notninecount += numnotnine
            ninecount += numnine
        else:
            total_idxs = list(idxs_notnine[notninecount:notninecount + bag_size])
            bag_idxs.append(total_idxs)
            bag_label.append(0)
            notninecount += bag_size
            
    return bag_idxs, bag_label
        
    
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

orig_x_shape = len(x_train)//2
x_val = x_train[orig_x_shape:]
y_val = y_train[orig_x_shape:]

x_train = x_train[:orig_x_shape]
y_train = y_train[:orig_x_shape]

# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

print('../MNIST/X_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])

sparsity = 1/10
bag_size = 20
num_bags = 250

test_bag_idxs, test_bag_label = create_bags(x_test, y_test, sparsity, bag_size, num_bags)

actual_x_test = []
actual_y_test = []
X_test = []
for tbi in test_bag_idxs:
    inner = []
    for i in tbi:
        inner.append(x_test[i])
        actual_x_test.append(x_test[i])
        actual_y_test.append(y_test[i])
    X_test.append(inner)

joblib.dump(np.array(X_test), '../MNIST/X_test_%d_%d_%d'%((int(sparsity*100), bag_size, num_bags)))
joblib.dump(test_bag_label, '../MNIST/y_test_%d_%d_%d'%((int(sparsity*100), bag_size, num_bags)))

joblib.dump(np.array(actual_x_test), '../MNIST/actual_X_test_%d_%d_%d'%((int(sparsity*100), bag_size, num_bags)))
joblib.dump(np.array(actual_y_test), '../MNIST/actual_y_test_%d_%d_%d'%((int(sparsity*100), bag_size, num_bags)))

# print(np.array(X_test).shape, len(y_test))

all_bag_sizes = [100, 1000, 10000]
for i, num_bags in enumerate(all_bag_sizes):
    if i == 0:
        actual_x_train = []
        actual_x_val = []
        actual_y_train = []
        actual_y_val = []
        X_train = []
        X_val = []
        new_y_train = []
        new_y_val = []
        num_bags_left = num_bags
    else:
        actual_x_train = list(joblib.load('../MNIST/actual_X_train_%d_%d_%d'%((int(sparsity*100), bag_size, all_bag_sizes[i - 1]))))
        actual_x_val = list(joblib.load('../MNIST/actual_X_val_%d_%d_%d'%((int(sparsity*100), bag_size, all_bag_sizes[i - 1]))))
        actual_y_train = list(joblib.load('../MNIST/actual_y_train_%d_%d_%d'%((int(sparsity*100), bag_size, all_bag_sizes[i - 1]))))
        actual_y_val = list(joblib.load('../MNIST/actual_y_val_%d_%d_%d'%((int(sparsity*100), bag_size, all_bag_sizes[i - 1]))))
        X_train = list(joblib.load('../MNIST/X_train_%d_%d_%d'%((int(sparsity*100), bag_size, all_bag_sizes[i - 1]))))
        X_val = list(joblib.load('../MNIST/X_val_%d_%d_%d'%((int(sparsity*100), bag_size, all_bag_sizes[i - 1]))))
        new_y_train = list(joblib.load('../MNIST/y_train_%d_%d_%d'%((int(sparsity*100), bag_size, all_bag_sizes[i - 1]))))
        new_y_val = list(joblib.load('../MNIST/y_val_%d_%d_%d'%((int(sparsity*100), bag_size, all_bag_sizes[i - 1]))))
    
        num_bags_left = num_bags - all_bag_sizes[i - 1]
        
    train_bag_idxs, train_bag_label = create_bags(x_train, y_train, sparsity, bag_size, num_bags_left)
    val_bag_idxs, val_bag_label = create_bags(x_val, y_val, sparsity, bag_size, num_bags_left)

    new_y_train.extend(train_bag_label)
    new_y_val.extend(val_bag_label)

    for tbi in train_bag_idxs:
        inner = []
        for i in tbi:
            inner.append(x_train[i])
            actual_x_train.append(x_train[i])
            actual_y_train.append(y_train[i])
        X_train.append(inner)

    for tbi in val_bag_idxs:
        inner = []
        for i in tbi:
            inner.append(x_val[i])
            actual_x_val.append(x_val[i])
            actual_y_val.append(y_val[i])
        X_val.append(inner)

    joblib.dump(np.array(X_train), '../MNIST/X_train_%d_%d_%d'%((int(sparsity*100), bag_size, num_bags)))
    joblib.dump(new_y_train, '../MNIST/y_train_%d_%d_%d'%((int(sparsity*100), bag_size, num_bags)))

    joblib.dump(np.array(X_val), '../MNIST/X_val_%d_%d_%d'%((int(sparsity*100), bag_size, num_bags)))
    joblib.dump(new_y_val, '../MNIST/y_val_%d_%d_%d'%((int(sparsity*100), bag_size, num_bags)))

    joblib.dump(np.array(actual_x_train), '../MNIST/actual_X_train_%d_%d_%d'%((int(sparsity*100), bag_size, num_bags)))
    joblib.dump(np.array(actual_y_train), '../MNIST/actual_y_train_%d_%d_%d'%((int(sparsity*100), bag_size, num_bags)))

    joblib.dump(np.array(actual_x_val), '../MNIST/actual_X_val_%d_%d_%d'%((int(sparsity*100), bag_size, num_bags)))
    joblib.dump(np.array(actual_y_val), '../MNIST/actual_y_val_%d_%d_%d'%((int(sparsity*100), bag_size, num_bags)))

    print(np.array(X_train).shape, np.array(X_val).shape, len(new_y_train), len(new_y_val))

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)
MNIST/X_train shape: (30000, 28, 28)
Number of images in x_train 30000
Number of images in x_test 10000
(100, 20, 28, 28) (100, 20, 28, 28) 100 100
(1000, 20, 28, 28) (1000, 20, 28, 28) 1000 1000
(10000, 20, 28, 28) (10000, 20, 28, 28) 10000 10000


In [2]:
print(joblib.load('../MNIST/actual_X_train_%d_%d_%d'%((int(sparsity*100), bag_size, num_bags))).shape)

(200000, 28, 28)


In [3]:
print(joblib.load('../MNIST/X_train_%d_%d_%d'%((int(sparsity*100), bag_size, num_bags))).shape)

(10000, 20, 28, 28)


In [4]:
print(actual_y_train[0:10])
print(actual_y_train[10:20])

[3, 4, 2, 5, 0, 7, 5, 5, 8, 8]
[2, 0, 1, 2, 7, 6, 3, 8, 9, 9]
